# 컬럼별 지오코딩

In [34]:
import requests
import pandas as pd
import time
from tqdm import tqdm
import os

tqdm.pandas()

# CSV 파일 불러오기
file_path = "C:/Users/jeong/Desktop/작업파일.csv" #파일명
data = pd.read_csv(file_path, encoding='utf-8')

# ✅ [추가된 부분] 종별코드명으로 필터링
data = data[data['종별코드명'].isin(['병원','의원','한방병원'])]
print(f"필터링된 데이터 개수: {len(data)}개")

data.head(2)

필터링된 데이터 개수: 36405개


,암호화YKIHO코드,요양기관명,종별코드,종별코드명,시도코드,시도명,시군구코드,시군구명,읍면동,우편번호,...,병원URL,개설일자,의사총수,일반의 의사수,인턴 의사수,레지던트 의사수,전문의 의사수,좌표정보(X),좌표정보(Y),Unnamed: 21
372,JDQ4MTYyMiM2MSMkMSMkMiMkOTkkMzgxMzUxIzExIyQxIy...,(신)제일병원,21,병원,340000,충남,340400,보령시,NaN,33459.0,...,NaN,2005-01-27,4,0,0,0,4,126.593625,36.343682,4.418054e+09
373,JDQ4MTYyMiM4MSMkMSMkNCMkOTkkMzgxMzUxIzExIyQxIy...,(의) 내경의료재단 울산제일병원,21,병원,260000,울산,260001,울산남구,NaN,44623.0,...,http://www.ujh.co.kr,2000-01-01,9,1,0,0,8,129.307013,35.548283,3.114051e+09


In [35]:
# 2. 불러온 파일명을 기반으로 저장할 파일명 만들기
base_name = os.path.splitext(file_path)[0]
output_file_path = f"{base_name}병원_의원_한방병원_지오코딩.csv"


# 주소를 좌표로 변환하는 함수 (이전과 동일)
def get_coordinates(address):
    time.sleep(0.01)
    apiurl = "https://api.vworld.kr/req/address?"
    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": address,
        "format": "json",
        "type": "road",
        "key": "B0586898-546F-3E6D-9D49-D569D3262EEE" #api
    }
    try:
        response = requests.get(apiurl, params=params)
        if response.status_code == 200:
            json_data = response.json()
            if json_data['response']['status'] == 'OK':
                point_info = json_data['response']['result']['point']
                return point_info['x'], point_info['y']
    except Exception as e:
        print(f"주소 변환 중 에러 발생: {address}, 오류: {e}")
    return None, None

# 함수를 '주소' 컬럼에 적용하고 결과를 새 컬럼에 저장
data[['X', 'Y']] = data['주소'].progress_apply(lambda addr: pd.Series(get_coordinates(addr)))

# 결과 확인
print(data.head())

# 3. 직접 지정한 파일명 대신, 위에서 만든 변수(output_file_path)를 사용
data.to_csv(output_file_path, index=False, encoding='utf-8-sig')

print(f"\n지오코딩 작업이 완료되어 '{output_file_path}' 파일로 저장되었습니다.")

 94%|█████████████████████████████████████████████████████████████████████▍    | 34160/36405 [1:06:36<02:46, 13.51it/s]

주소 변환 중 에러 발생: 경기도 부천시 소사로 180 (소사본동), 오류: ('Connection aborted.', ConnectionAbortedError(10053, '현재 연결은 사용자의 호스트 시스템의 소프트웨어의 의해 중단되었습니다', None, 10053, None))
주소 변환 중 에러 발생: 경기도 남양주시 금곡로 72 303호 (금곡동, 센타플라자), 오류: HTTPSConnectionPool(host='api.vworld.kr', port=443): Max retries exceeded with url: /req/address?service=address&request=getcoord&crs=epsg%3A4326&address=%EA%B2%BD%EA%B8%B0%EB%8F%84+%EB%82%A8%EC%96%91%EC%A3%BC%EC%8B%9C+%EA%B8%88%EA%B3%A1%EB%A1%9C+72+303%ED%98%B8+%28%EA%B8%88%EA%B3%A1%EB%8F%99%2C+%EC%84%BC%ED%83%80%ED%94%8C%EB%9D%BC%EC%9E%90%29&format=json&type=road&key=B0586898-546F-3E6D-9D49-D569D3262EEE (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000002CB2B45CE10>: Failed to resolve 'api.vworld.kr' ([Errno 11001] getaddrinfo failed)"))
주소 변환 중 에러 발생: 전라남도 화순군 화순읍 자치샘로 36 3층, 오류: HTTPSConnectionPool(host='api.vworld.kr', port=443): Max retries exceeded with url: /req/address?service=address&request=getcoord&crs=epsg%3A4326&address=%EC%A0%84

 94%|█████████████████████████████████████████████████████████████████████▍    | 34176/36405 [1:06:37<01:11, 31.34it/s]

주소 변환 중 에러 발생: 전라북도 전주시 덕진구 전주천동로 506 (금암동), 오류: HTTPSConnectionPool(host='api.vworld.kr', port=443): Max retries exceeded with url: /req/address?service=address&request=getcoord&crs=epsg%3A4326&address=%EC%A0%84%EB%9D%BC%EB%B6%81%EB%8F%84+%EC%A0%84%EC%A3%BC%EC%8B%9C+%EB%8D%95%EC%A7%84%EA%B5%AC+%EC%A0%84%EC%A3%BC%EC%B2%9C%EB%8F%99%EB%A1%9C+506+%28%EA%B8%88%EC%95%94%EB%8F%99%29&format=json&type=road&key=B0586898-546F-3E6D-9D49-D569D3262EEE (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000002CB2591E350>: Failed to resolve 'api.vworld.kr' ([Errno 11001] getaddrinfo failed)"))
주소 변환 중 에러 발생: 전라북도 군산시 나운로 48 301호 (나운동, 소원빌딩), 오류: HTTPSConnectionPool(host='api.vworld.kr', port=443): Max retries exceeded with url: /req/address?service=address&request=getcoord&crs=epsg%3A4326&address=%EC%A0%84%EB%9D%BC%EB%B6%81%EB%8F%84+%EA%B5%B0%EC%82%B0%EC%8B%9C+%EB%82%98%EC%9A%B4%EB%A1%9C+48+301%ED%98%B8+%28%EB%82%98%EC%9A%B4%EB%8F%99%2C+%EC%86%8C%EC%9B%90%EB%B9%8C%EB%94%A9

 94%|█████████████████████████████████████████████████████████████████████▌    | 34192/36405 [1:06:37<00:46, 47.83it/s]

주소 변환 중 에러 발생: 경기도 남양주시 진건읍 사릉로 400 1층, 2층 (노아쇼핑), 오류: HTTPSConnectionPool(host='api.vworld.kr', port=443): Max retries exceeded with url: /req/address?service=address&request=getcoord&crs=epsg%3A4326&address=%EA%B2%BD%EA%B8%B0%EB%8F%84+%EB%82%A8%EC%96%91%EC%A3%BC%EC%8B%9C+%EC%A7%84%EA%B1%B4%EC%9D%8D+%EC%82%AC%EB%A6%89%EB%A1%9C+400+1%EC%B8%B5%2C+2%EC%B8%B5+%28%EB%85%B8%EC%95%84%EC%87%BC%ED%95%91%29&format=json&type=road&key=B0586898-546F-3E6D-9D49-D569D3262EEE (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000002CB21E62AD0>: Failed to resolve 'api.vworld.kr' ([Errno 11001] getaddrinfo failed)"))
주소 변환 중 에러 발생: 경기도 여주시 가남읍 태평로 25 (가남읍), 오류: HTTPSConnectionPool(host='api.vworld.kr', port=443): Max retries exceeded with url: /req/address?service=address&request=getcoord&crs=epsg%3A4326&address=%EA%B2%BD%EA%B8%B0%EB%8F%84+%EC%97%AC%EC%A3%BC%EC%8B%9C+%EA%B0%80%EB%82%A8%EC%9D%8D+%ED%83%9C%ED%8F%89%EB%A1%9C+25+%28%EA%B0%80%EB%82%A8%EC%9D%8D%29&format=json&type

 94%|█████████████████████████████████████████████████████████████████████▌    | 34199/36405 [1:06:40<05:36,  6.56it/s]

주소 변환 중 에러 발생: 경기도 파주시 중앙로 325 (금촌동), 오류: HTTPSConnectionPool(host='api.vworld.kr', port=443): Max retries exceeded with url: /req/address?service=address&request=getcoord&crs=epsg%3A4326&address=%EA%B2%BD%EA%B8%B0%EB%8F%84+%ED%8C%8C%EC%A3%BC%EC%8B%9C+%EC%A4%91%EC%95%99%EB%A1%9C+325+%28%EA%B8%88%EC%B4%8C%EB%8F%99%29&format=json&type=road&key=B0586898-546F-3E6D-9D49-D569D3262EEE (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002CB2F9882D0>: Failed to establish a new connection: [WinError 10053] 현재 연결은 사용자의 호스트 시스템의 소프트웨어의 의해 중단되었습니다'))


100%|██████████████████████████████████████████████████████████████████████████| 36405/36405 [1:11:17<00:00,  8.51it/s]


                                            암호화YKIHO코드              요양기관명  \
372  JDQ4MTYyMiM2MSMkMSMkMiMkOTkkMzgxMzUxIzExIyQxIy...            (신)제일병원   
373  JDQ4MTYyMiM4MSMkMSMkNCMkOTkkMzgxMzUxIzExIyQxIy...  (의) 내경의료재단 울산제일병원   
374  JDQ4MTYyMiM4MSMkMSMkNCMkOTkkMzgxMzUxIzIxIyQyIy...   (의)경빈의료재단 김해센텀병원   
375  JDQ4MTYyMiM4MSMkMSMkMCMkOTkkMzgxMzUxIzExIyQxIy...     (의)서봉의료재단 보강병원   
376  JDQ4MTAxMiM1MSMkMSMkMCMkOTkkMzgxMzUxIzExIyQxIy...      (의)서일의료재단기장병원   

     종별코드 종별코드명    시도코드 시도명   시군구코드   시군구명  읍면동     우편번호  ... 의사총수 일반의 의사수  \
372    21    병원  340000  충남  340400    보령시  NaN  33459.0  ...    4       0   
373    21    병원  260000  울산  260001   울산남구  NaN  44623.0  ...    9       1   
374    21    병원  380000  경남  380100    김해시  NaN  50948.0  ...    6       0   
375    21    병원  230000  대구  230007  대구달서구  NaN  42801.0  ...   18       0   
376    21    병원  210000  부산  210100  부산기장군  NaN  46072.0  ...   10       0   

    인턴 의사수 레지던트 의사수  전문의 의사수     좌표정보(X)    좌표정보(Y)   Unnamed: 21  \

# X, Y 넓값 확인

In [36]:
# 지오코딩된 CSV파일 불러오기
file_path = r"C:\Users\jeong\Desktop\작업파일병원_의원_한방병원_지오코딩.csv" #파일명
data = pd.read_csv(file_path, encoding='utf-8')

# X, Y 컬럼의 결측치 개수 확인
print(data[['X', 'Y']].isnull().sum())

#결측치 행 확인
null_rows = data[data['X'].isnull() | data['Y'].isnull()]
print(null_rows[['주소', '종별코드명']])

X    432
Y    432
dtype: int64
                                             주소 종별코드명
4                   부산광역시 기장군 기장읍 대청로72번길 6-6 6    병원
47     서울특별시 관악구 남부순환로 1796 -1,2~5층 (봉천동, 삼호빌딩)    병원
51               서울특별시 강동구 천호대로 1043-1043 (천호동)    병원
90                       경상남도 진주시 금산면 송백로 46 46    병원
91                충청북도 청주시 상당구 남일면 단재로 635-0 ()    병원
...                                         ...   ...
35323               충청남도 홍성군 홍성읍 충서로 1245 (홍성읍)    의원
35469                   경기도 화성시 마도면 화성로 741 741    의원
35561         서울특별시 중랑구 사가정로 389 KFC 4,5층 (면목동)    의원
35902                 서울특별시 도봉구 도당로 51-51 (방학동)  한방병원
36151                  경기도 시흥시 삼미시장1길 5-1 (신천동)  한방병원

[432 rows x 2 columns]


In [37]:
# 좌표가 정상 범위 안에 있는지 확인
invalid = data[
    (data['Y'] < 33) | (data['Y'] > 39) |
    (data['X'] < 124) | (data['X'] > 132)
]
print(f"비정상 좌표 개수: {len(invalid)}개")

비정상 좌표 개수: 0개


In [38]:
# X, Y 컬럼의 결측치 확인
null_rows = data[data['X'].isnull() | data['Y'].isnull()]

# 널값이 있는 행 수 확인
print(f"⚠️ X 또는 Y가 널인 행 수: {len(null_rows)}개")

# 널값이 없는 행만 남기기
valid_rows = data[data['X'].notnull() & data['Y'].notnull()]
print(f"✅ 좌표가 있는 행 수: {len(valid_rows)}개")

# 널값 있는 행 별도 CSV로 저장
null_file_path = "C:/Users/jeong/Desktop/병원_의원_한방병원_지오코딩_널값.csv"
null_rows.to_csv(null_file_path, index=False, encoding='utf-8-sig')
print(f"널값 행 저장 완료: {null_file_path}")

# 널값 없는 행 저장 (기존 지오코딩 파일 덮어쓰기 가능)
output_file_path = "C:/Users/jeong/Desktop/병원_의원_한방병원_지오코딩_널값제외.csv"
valid_rows.to_csv(output_file_path, index=False, encoding='utf-8-sig')
print(f"널값 제거 후 좌표 있는 행 저장 완료: {output_file_path}")


⚠️ X 또는 Y가 널인 행 수: 432개
✅ 좌표가 있는 행 수: 35973개
널값 행 저장 완료: C:/Users/jeong/Desktop/병원_의원_한방병원_지오코딩_널값.csv
널값 제거 후 좌표 있는 행 저장 완료: C:/Users/jeong/Desktop/병원_의원_한방병원_지오코딩_널값제외.csv


# 파일 합치기


In [41]:
#1. 파일 불러오기
df_null_1 = pd.read_csv(r"C:\Users\jeong\Desktop\상급종합_종합병원_지오코딩_널값제외.csv")
df_null_2 = pd.read_csv(r"C:\Users\jeong\Desktop\병원_의원_한방병원_지오코딩_널값제외.csv")

# 2. 두 데이터프레임을 아래 방향(행 방향)으로 합치기
# axis=0: 행(Row) 방향으로 합치기 (아래 방향)
# ignore_index=True: 합친 후 인덱스를 0부터 새로 재설정
df_combined_null = pd.concat([df_null_1, df_null_2], axis=0, ignore_index=True)

#3. 파일 저장
output_file_path = "C:/Users/jeong/Desktop/지오코딩_널값제외종합.csv"
df_combined_null.to_csv(output_file_path, index=False, encoding='utf-8-sig')